Ideas:

* Add FFT channels to CNN (Tried, does not make much difference)
* Finetune CNN (with SGD slow learning rate)
* 5-fold CNN
* Extract Features from CNN (before FC) and do XGB
* TTA (tried, made it better)
* More augmenting, additional 45, 135, 315 degrees
* More augmenting, random rotations and flips
* Predict test data and train with test
* Train on all of the training data (no train-val split)
* Try a different combination of combine predictions
* Fine-tune on pre-trained models (Get rid of some top layers because input size is small)

Ideas2:
* Add more aug (h_flip + rot90, etc.)
* Add more layers to CNN
* Different CNN architecture

In [1]:
import numpy as np
import pandas as pd
from skimage import transform
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from keras import losses, optimizers, callbacks
import xgboost as xgb
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_validate, train_test_split, StratifiedKFold, KFold
from scipy import fftpack
from itertools import product
import matplotlib.pyplot as plt
%matplotlib inline
RANDOM_SEED = 43
np.random.seed(RANDOM_SEED)

Using TensorFlow backend.
/home/can/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Create Dataset 

In [2]:
# Train
df = pd.read_json('./data/train.json')
X, y = [], []
for im_band1, im_band2, label in zip(df['band_1'], df['band_2'], df['is_iceberg']):
    im_band1 = np.array(im_band1).reshape(75, 75, 1)
    im_band2 = np.array(im_band2).reshape(75, 75, 1)    
    # Preprocess
    # - Zero mean
    im_band1 -= np.mean(im_band1)
    im_band2 -= np.mean(im_band2)
    # - Normalize
    im_band1 /= np.std(im_band1)
    im_band2 /= np.std(im_band2)    
    im = np.concatenate([im_band1, im_band2], axis=2)
    X.append(im)
    y.append(label)    
X = np.array(X)
y = np.array(y)
print 'X.shape:', X.shape
print 'y.shape:', y.shape

X.shape: (1604, 75, 75, 2)
y.shape: (1604,)


## Train - Val Split

In [3]:
N_SPLITS = 5
MODEL_NUMBER = 5
skf = StratifiedKFold(n_splits=N_SPLITS, random_state=RANDOM_SEED, shuffle=True)
cv = list(skf.split(X, y))

In [4]:
train_i, val_i = cv[MODEL_NUMBER - 1]
X_train, y_train = X[train_i], y[train_i]
X_val, y_val = X[val_i], y[val_i]
print 'X_train.shape:', X_train.shape
print 'y_train.shape:', y_train.shape
print 'X_val.shape:', X_val.shape
print 'y_val.shape:', y_val.shape
print 'np.mean(y_train):', np.mean(y_train)
print 'np.mean(y_val):', np.mean(y_val)

X_train.shape: (1284, 75, 75, 2)
y_train.shape: (1284,)
X_val.shape: (320, 75, 75, 2)
y_val.shape: (320,)
np.mean(y_train): 0.469626168224
np.mean(y_val): 0.46875


In [5]:
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)
# print 'X_train.shape:', X_train.shape
# print 'X_val.shape:', X_val.shape
# print 'y_train.shape:', y_train.shape
# print 'y_val.shape:', y_val.shape

## Data Augmentation

In [6]:
def bypass(x):
    return x

# Flips
def hflip(x):
    return x[:, :, ::-1, :]

def vflip(x):
    return x[:, ::-1, :, :]

def hvflip(x):
    return hflip(vflip(x))

# Rotations
def rot90(x):
    return np.concatenate([np.expand_dims(transform.rotate(im, 90), axis=0) for im in x], axis=0)

def rot180(x):
    return np.concatenate([np.expand_dims(transform.rotate(im, 180), axis=0) for im in x], axis=0)

def rot270(x):
    return np.concatenate([np.expand_dims(transform.rotate(im, 270), axis=0) for im in x], axis=0)

# Combinations
def merge_f(t):
    """
    Combines functions in the tuple
    t: (tuple)
    """
    f1, f2 = t
    return lambda x: f1(f2(x))

# Flips, Rotations
flips = [hflip, vflip, hvflip]
rots = [rot90, rot180, rot270]
# Combine
flips_and_rots = [merge_f(t) for t in list(product(flips, rots))]
rots_and_flips = [merge_f(t) for t in list(product(rots, flips))]


aug_funcs = [bypass] + flips + rots + flips_and_rots + rots_and_flips
print 'len(aug_funcs):', len(aug_funcs)

len(aug_funcs): 25


In [7]:
# Train
X_train = np.concatenate([func(X_train) for func in aug_funcs], axis=0)
y_train = np.concatenate([y_train] * len(aug_funcs))

# Validation
X_val = np.concatenate([func(X_val) for func in aug_funcs], axis=0)
y_val = np.concatenate([y_val] * len(aug_funcs))

# 
print 'X_train.shape:', X_train.shape
print 'y_train.shape:', y_train.shape
print 'X_val.shape:', X_val.shape
print 'y_val.shape:', y_val.shape

X_train.shape: (32100, 75, 75, 2)
y_train.shape: (32100,)
X_val.shape: (8000, 75, 75, 2)
y_val.shape: (8000,)


# Training

In [8]:
def get_model(input_shape):
    # Architecture
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3),
                     activation='relu',
                     input_shape=input_shape))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.33))
    model.add(Conv2D(256, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))   
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    
    # Compile the model
    loss = losses.binary_crossentropy
    optimizer = optimizers.Adam()
    metrics = ['accuracy']
    model.compile(loss=loss, optimizer=optimizer, metrics=metrics)
    
    #     
    return model

In [9]:
model = get_model(input_shape=(75, 75, 2))

In [10]:
# Callbacks
def get_lr(epoch):
    lr = (np.random.rand() * 1e-3 + 1e-7)
    print 'lr:', lr
    return lr
MODEL_PATH = './models/model7/model' + str(MODEL_NUMBER) + '.h5'
m_q = 'val_loss'
model_path = MODEL_PATH
check_pt = callbacks.ModelCheckpoint(filepath=model_path, monitor=m_q, save_best_only=True, verbose=1)
early_stop = callbacks.EarlyStopping(patience=6, monitor=m_q, verbose=1)
reduce_lr = callbacks.ReduceLROnPlateau(patience=1, factor=0.5, monitor=m_q, verbose=1)
schedule_lr = callbacks.LearningRateScheduler(get_lr)
callback_list = [check_pt, early_stop, reduce_lr]

model.fit(X_train, y_train, validation_data=(X_val, y_val), callbacks=callback_list, epochs=200)

Train on 32100 samples, validate on 8000 samples
Epoch 1/200
32100/32100 [==============================] - 20s 624us/step - loss: 0.3528 - acc: 0.8332 - val_loss: 0.2463 - val_acc: 0.8928
Epoch 2/200
32100/32100 [==============================] - 19s 593us/step - loss: 0.2068 - acc: 0.9138 - val_loss: 0.2100 - val_acc: 0.9155
Epoch 3/200
32100/32100 [==============================] - 19s 591us/step - loss: 0.1859 - acc: 0.9255 - val_loss: 0.2249 - val_acc: 0.9134
Epoch 4/200
32032/32100 [============================>.] - ETA: 0s - loss: 0.1416 - acc: 0.9430Epoch 00004: val_loss did not improve

Epoch 00004: reducing learning rate to 0.000500000023749.
32100/32100 [==============================] - 19s 600us/step - loss: 0.1414 - acc: 0.9431 - val_loss: 0.2499 - val_acc: 0.9107
Epoch 5/200
32032/32100 [============================>.] - ETA: 0s - loss: 0.0938 - acc: 0.9625Epoch 00005: val_loss did not improve

Epoch 00005: reducing learning rate to 0.000250000011874.
32100/32100 [======

In [11]:
# # Finetune
# loss = losses.binary_crossentropy
# optimizer = optimizers.SGD(lr=1e-4)
# metrics = ['accuracy']
# model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

In [ ]:
# m_q = 'val_loss'
# model_path = MODEL_PATH
# check_pt = callbacks.ModelCheckpoint(filepath=model_path, monitor=m_q, save_best_only=True, verbose=1)
# early_stop = callbacks.EarlyStopping(patience=5, monitor=m_q, verbose=1)
# reduce_lr = callbacks.ReduceLROnPlateau(patience=2, factor=0.33, monitor=m_q, verbose=1)
# callback_list = [check_pt, early_stop, reduce_lr]

# model.fit(X_train, y_train, validation_data=(X_val, y_val), callbacks=callback_list, epochs=200)

# Predict Test

In [11]:
# Load test data
df = pd.read_json('./data/test.json')
X_test, y_test = [], []
for im_band1, im_band2 in zip(df['band_1'], df['band_2']):
    im_band1 = np.array(im_band1).reshape(75, 75, 1)
    im_band2 = np.array(im_band2).reshape(75, 75, 1)    
    # Preprocess - zero mean
    im_band1 -= np.mean(im_band1)
    im_band2 -= np.mean(im_band2)
    # Preprocess - normalize
    im_band1 /= np.std(im_band1)
    im_band2 /= np.std(im_band2)    
    im = np.concatenate([im_band1, im_band2], axis=2)
    X_test.append(im)    
X_test = np.array(X_test)
print 'X_test.shape:', X_test.shape

X_test.shape: (8424, 75, 75, 2)


In [12]:
y_test_p = 0
# weights = [0.3, 0.4 / 3, 0.3, 0.4 / 3, 0.4 / 3]
weights = [11, 10, 14, 4, 11]
for i, w in zip(range(5), weights):
    print i
    # Load the model
    MODEL_PATH = './models/model4/model' + str(i + 1) + '.h5'
    model = load_model(MODEL_PATH)
    # predict - tta    
    for func in aug_funcs:
        y_test_p += model.predict(func(X_test), verbose=1).flatten() * w
# y_test_p = y_test_p / (len(aug_funcs) * 5.0)
y_test_p = y_test_p / (len(aug_funcs) * float(sum(weights)))

0
8424/8424 [==============================] - 2s 197us/step
1
8424/8424 [==============================] - 1s 175us/step
2
8424/8424 [==============================] - 2s 242us/step
3
8424/8424 [==============================] - 2s 184us/step
4
8424/8424 [==============================] - 2s 231us/step


In [13]:
df_sub = pd.DataFrame()
df_sub['id'] = df['id']
df_sub['is_iceberg'] = y_test_p.flatten()
df_sub.to_csv('./submissions/sub17.csv', index=False)